# Load parameters

In [ ]:
sample = SAMPLE
file = BARCARD_OVERLAP_TSV

# Import libraries and define functions

In [ ]:
import os
import pandas as pd
import seaborn as sns
import glob
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def histogram(array, nbins=100):
    """
    Draw histogram from distribution and identify centers.
    Parameters
    ---------
    array: `class::np.array`
            Scores distribution
    nbins: int
            Number of bins to use in the histogram
    Return
    ---------
    float
            Histogram values and bin centers.
    """
    array = array.ravel().flatten()
    hist, bin_edges = np.histogram(array, bins=nbins, range=None)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2.0
    return hist, bin_centers


def threshold_otsu(array, nbins=100):
    """
    Apply Otsu threshold on topic-region distributions [Otsu, 1979].
    Parameters
    ---------
    array: `class::np.array`
            Array containing the region values for the topic to be binarized.
    nbins: int
            Number of bins to use in the binarization histogram
    Return
    ---------
    float
            Binarization threshold.
    Reference
    ---------
    Otsu, N., 1979. A threshold selection method from gray-level histograms. IEEE transactions on systems, man, and
    cybernetics, 9(1), pp.62-66.
    """
    hist, bin_centers = histogram(array, nbins)
    hist = hist.astype(float)
    # Class probabilities for all possible thresholds
    weight1 = np.cumsum(hist)
    weight2 = np.cumsum(hist[::-1])[::-1]
    # Class means for all possible thresholds
    mean1 = np.cumsum(hist * bin_centers) / weight1
    mean2 = (np.cumsum((hist * bin_centers)[::-1]) / weight2[::-1])[::-1]
    # Clip ends to align class 1 and class 2 variables:
    # The last value of ``weight1``/``mean1`` should pair with zero values in
    # ``weight2``/``mean2``, which do not exist.
    variance12 = weight1[:-1] * weight2[1:] * (mean1[:-1] - mean2[1:]) ** 2
    idx = np.argmax(variance12)
    threshold = bin_centers[:-1][idx]
    return threshold

# Calculate threshold

In [ ]:
threshold_min = 0.02

print(sample)
df = pd.read_csv(file, header=0, sep="\t")

df = df.sort_values(by="jaccard", ascending=False)[:1000000]
df.reset_index(inplace=True, drop=True)
threshold = threshold_dict[sample]
threshold_rank = threshold_rank_dict[sample]

print(f"\tthreshold: {threshold}")
print(f"\tnpairs_merged: {threshold_rank}")

f, ax = plt.subplots(1, 1)
sns.lineplot(data=df, x=range(len(df)), y="jaccard", ax=ax)
# ax.axhline(y=threshold, xmin=0, xmax=10000000)
ax.axvline(x=threshold_rank, ymin=0.0001, ymax=1)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_title(sample)
ax.set_title(
    f"{sample}, threshold {round(threshold, 3)}, {threshold_rank} pairs merged"
)
plt.show()
plt.savefig(
    f"{sample}.barcard_kneeplot.png",
    dpi=300,
    facecolor="white",
)
df.iloc[:threshold_rank].to_csv(
    f"{sample}.barcard.overlap.otsu_filtered.tsv", sep="\t", index=False
)